In [1]:
# Plan
# Apply YOLOv4 to any image data
# Can use COCO dataset or Multi-Moments in Time


# Changed from 2.2.0 to 2.3.0 due to Tensorflow incompatability
!pip install --no-cache-dir https://github.com/sicara/tf2-yolov4/archive/master.zip
!pip install tensorflow==2.3.0



     - 2.7 MB 1.4 MB/s
     |████████████████████████████████| 1.1 MB 5.0 MB/s 
  Created wheel for tf2-yolov4: filename=tf2_yolov4-0.1.0-py3-none-any.whl size=20158 sha256=b7f61b7f23a3750931b00325942e279de488f16a8a5af712e05f8befc0abfb63
  Stored in directory: /tmp/pip-ephem-wheel-cache-fcrktwzr/wheels/56/ba/d4/430cf7011ba67118d957b8d5b3d46b312eca092cde0ba9638c
Successfully built tf2-yolov4
     |████████████████████████████████| 320.4 MB 44 kB/s 
     |████████████████████████████████| 2.9 MB 38.1 MB/s 
     |████████████████████████████████| 20.1 MB 40.3 MB/s 
     |████████████████████████████████| 459 kB 38.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting u

In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!cd drive/MyDrive/YTVidObjectDectector/frames/Train/

[YOLOv4 Detection Example](https://github.com/sicara/tf2-yolov4/blob/master/notebooks/YoloV4_Dectection_Example.ipynb)

[Deep Feature Flow For Video Recognition](https://github.com/msracver/Deep-Feature-Flow/blob/master/README.md)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT" -O yolov4.weights
!rm -rf /tmp/cookies.txt
!ls -la ./yolov4.weights

In [ ]:
!convert-darknet-weights yolov4.weights -o yolov4.h5
!ls -la ./yolov4.h5

In [ ]:
import tensorflow as tf

from tf2_yolov4.anchors import YOLOV4_ANCHORS
from tf2_yolov4.model import YOLOv4

HEIGHT, WIDTH = (640, 960)

In [ ]:
!wget https://github.com/sicara/tf2-yolov4/raw/master/notebooks/images/cars.jpg

from IPython.display import Image
Image('./cars.jpg')

In [ ]:
image = tf.io.read_file("./cars.jpg")
image = tf.image.decode_image(image)
image = tf.image.resize(image, (HEIGHT, WIDTH))
images = tf.expand_dims(image, axis=0) / 255.0

In [ ]:
model = YOLOv4(
    input_shape=(HEIGHT, WIDTH, 3),
    anchors=YOLOV4_ANCHORS,
    num_classes=80,
    training=False,
    yolo_max_boxes=100,
    yolo_iou_threshold=0.5,
    yolo_score_threshold=0.5,
    
)

model.load_weights("./yolov4.h5")
model.summary()

In [ ]:
boxes, scores, classes, valid_detections = model.predict(images)

In [ ]:
# COCO classes
CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]
# Change classes to: person, teddy bear, random things, TV
video_CLASSES = [
                 'tv', 'person', 'bear', 'laptop', 'teddy bear', 'laptop', 'mouse', 'cellphone', 'cup'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

def plot_results(pil_img, boxes, scores, classes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()

    for (xmin, ymin, xmax, ymax), score, cl in zip(boxes.tolist(), scores.tolist(), classes.tolist()):
        if score > 0:
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=COLORS[cl % 6], linewidth=3))
          text = f'{CLASSES[cl]}: {score:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [ ]:
plot_results(
    images[0],
    boxes[0] * [WIDTH, HEIGHT, WIDTH, HEIGHT],
    scores[0],
    classes[0].astype(int),
)

In [ ]:
# For loop (?) over all images in training set
for i in .....:
  frame = tf.io.read_file(........)
  frame = tf.image.decode_image(frame)
  frame = tf.image.resize(frame, (HEIGHT, WIDTH))
  frames = tf.expand_dims(frame, axis=0) / 255.0

  boxes, scores, classes, valid_detections = model.predict(frames)
  plot_results(
    frames[0],
    boxes[0] * [WIDTH, HEIGHT, WIDTH, HEIGHT],
    scores[0],
    classes[0].astype(int))
)

In [ ]:
# Loss function and plot



# Precision, accuracy, recall metrics
# Make plots and summary table